# NAVER NEWS API를 활용한 수집

In [18]:
import os
import sys
import urllib.request
import requests
from dotenv import load_dotenv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


from transformers import PreTrainedTokenizerFast, BartModel, BartForConditionalGeneration
import torch

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [16]:
print("Vocab Size:", tokenizer.vocab_size)

print("Max Token ID in input_ids:", max(input_ids))

print(model.config.max_length)

Vocab Size: 30000
Max Token ID in input_ids: 29624
20


In [37]:
keyword= "마비노기"

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9가-힣 ]', '', text)
    return text.strip()

def is_similar(text1, text2, threshold=0.7):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    similarity = cosine_similarity(vectorizer[0], vectorizer[1])
    return similarity[0][0] > threshold




load_dotenv()
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

encText = urllib.parse.quote(f"{keyword}")
url = "https://openapi.naver.com/v1/search/news?display=20&sort=sim&query=" +encText

response = requests.get(url,headers={"X-Naver-Client-Id":client_id ,"X-Naver-Client-Secret":client_secret})
rescode = response.status_code
text_box=[]

if(rescode==200):
    response_body = response.json()
    link_list = [x['link'] for x in response_body['items'] if 'naver' in x['link']]
    driver =webdriver.Chrome()
    for link in link_list :
        driver.get(link)
        time.sleep(1)
        article = driver.find_elements(By.CLASS_NAME, "_article_content")
        text = [x.text for x in article][0]
        text = text.replace('\n', ' ')
        raw_input_ids = tokenizer.encode(text, truncation=True,max_length=1024)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
        summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=6,  min_length=24, max_length=256, bos_token_id=0, eos_token_id=1)
        result=tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        clean_result = clean_text(result)
        if not any(is_similar(clean_result, clean_text(existing)) for existing in text_box):
            text_box.append(clean_result)
    all_text = " ".join(text_box)
    driver.close()
    print(all_text)
    print(link_list)
else:
    print(f"에러가 발생했어요 ㅠ^ㅠ \n Error Code:{rescode}")


넷마블은 내달 20일 다중접속역할수행게임mmorpg rf 온라인 넥스트를 출시하며 넥슨은 마비노기 모바일 퍼스트 버서커 카잔 인조이inzoi rf 온라인 넥스트 등 신작들을 통해 국내 뿐만 아니라 글로벌 시장 공략을 시작한다 마비노기 모바일은 21년 전 원작을 탄생시킨 김동건 데브캣 대표가 직접 개발을 진두지휘한 마비노기 모바일이 3월 27일 정식 출시를 앞두고 사전 등록을 진행하고 있어 게임업계의 이목이 집중되고 있다 18일 게임슨코리아는 데브캣이 개발한 mmorpg 마비노기 모바일의 3월 출시를 앞두고 사전 예약 프로모션을 시작했으며 다음 달 27일 국내 앱 마켓에 정식 출시한다는 계획이다 넷마블은 마비노기 모바일 퍼스트 버서커 카잔 rf 온라인 넥스트 등 게임 팬들이 오랫동안 기다려온 신작들을 통해 국내 뿐만 아니라 글로벌 시장 공략을 시작한다 넷마블의 rf 온라인 넥스트를 시작으로 넥슨 마비노기 모바일 퍼스트 버서커 카잔 크래프톤의 인조이가 3월 국내 시장에 잇달아 출시된다 넥슨은이 마비노기 모바일 외에도 마비노기 영웅전의 후속작 개념인 싱글 플레이 기반 액션 게임 빈딕투스 디파잉 페이트 원작 마비노기의 언리얼엔진 교체 프로젝트인 마비노기 이터니티 등을 준비 중이다 넥슨 퍼스트 버서커 카잔과 크래프톤 인조이inzoi가 글로벌 시장을 겨냥한 k게임 유망 신작으로 3월 세계 무대에 출사표를 던진다 마비노기 지식재산권ip를 관장하는 마비노기 지식재산권ip를 관장하는 마비노기 디렉터 그리고 김동건 데브캣 대표는 마비노기의 장기적인 방향성과 비전을 전했다 넥슨은이 지난해 연간 매출 4조91억원엔화 4462억엔 영업이익 1조1157억원엔화 1242억 엔을 기록하며 국내 게임사 최초로 연간 매출 4조원 돌파에 성공했다고 13일 밝혔다 마비노기를 손 안에서 즐길 수 있는 마비노기 모바일은 마비노기 특유의 낭만과 감성을 계승하고 재해석해 새로운 모험의 경험을 제공할 예정이며 마비노기 모바일은 마비노기 특유의 낭만과 감성을 계승하고 재해석해 새로운 모험의 경험을 제공할 

In [15]:
print(summarize_text)

소속사 JYP엔터테인먼트는 걸그룹 트와이스 멤버를 대상으로 특정인이 망상에서 비롯된 부적절한 내용과 사진을 포함하여 아티스트 주변을 지속적으로 맴돌고 있다는 사실을 명시하는 편지들을 무분별하게 발송하고 있다며 법적 대응을 예고했다.


In [36]:
all_text

'넷마블은 내달 20일 다중접속역할수행게임mmorpg rf 온라인 넥스트를 출시하며 넥슨은 마비노기 모바일 퍼스트 버서커 카잔 인조이inzoi rf 온라인 넥스트 등 신작들을 통해 국내 뿐만 아니라 글로벌 시장 공략을 시작한다 마비노기 모바일은 21년 전 원작을 탄생시킨 김동건 데브캣 대표가 직접 개발을 진두지휘한 마비노기 모바일이 3월 27일 정식 출시를 앞두고 사전 등록을 진행하고 있어 게임업계의 이목이 집중되고 있다 18일 게임슨코리아는 데브캣이 개발한 mmorpg 마비노기 모바일의 3월 출시를 앞두고 사전 예약 프로모션을 시작했으며 다음 달 27일 국내 앱 마켓에 정식 출시한다는 계획이다 넷마블은 마비노기 모바일 퍼스트 버서커 카잔 rf 온라인 넥스트 등 게임 팬들이 오랫동안 기다려온 신작들을 통해 국내 뿐만 아니라 글로벌 시장 공략을 시작한다 넷마블의 rf 온라인 넥스트를 시작으로 넥슨 마비노기 모바일 퍼스트 버서커 카잔 크래프톤의 인조이가 3월 국내 시장에 잇달아 출시된다 넥슨은이 마비노기 모바일 외에도 마비노기 영웅전의 후속작 개념인 싱글 플레이 기반 액션 게임 빈딕투스 디파잉 페이트 원작 마비노기의 언리얼엔진 교체 프로젝트인 마비노기 이터니티 등을 준비 중이다 넥슨 퍼스트 버서커 카잔과 크래프톤 인조이inzoi가 글로벌 시장을 겨냥한 k게임 유망 신작으로 3월 세계 무대에 출사표를 던진다 마비노기 지식재산권ip를 관장하는 마비노기 지식재산권ip를 관장하는 마비노기 디렉터 그리고 김동건 데브캣 대표는 마비노기의 장기적인 방향성과 비전을 전했다 넥슨은이 지난해 연간 매출 4조91억원엔화 4462억엔 영업이익 1조1157억원엔화 1242억 엔을 기록하며 국내 게임사 최초로 연간 매출 4조원 돌파에 성공했다고 13일 밝혔다 마비노기를 손 안에서 즐길 수 있는 마비노기 모바일은 마비노기 특유의 낭만과 감성을 계승하고 재해석해 새로운 모험의 경험을 제공할 예정이며 마비노기 모바일은 마비노기 특유의 낭만과 감성을 계승하고 재해석해 새로운 모험의 경험을 제공할